In [234]:
import pandas as pd
import numpy as np
import string
from unicodedata import normalize

def mudaAnoUgestora(ano):
    if ano >= 2009 and ano <= 2012:
        return 2009
    elif ano >= 2013 and ano <= 2016:
        return 2013

def nomeUgestora(gestora):
    return gestora[24:].upper()

def upperPrefeito(nome):
    nome = normalize('NFKD', nome.decode('utf-8')).encode('ASCII','ignore')
    nome = nome.upper()
    return nome

def anoPublicacao(data):
    ano = int('20'+data[6:])
    if ano >= 2009 and ano <= 2012:
        return 2009
    elif ano >= 2013 and ano <= 2016:
        return 2013
    else:
        return 2008

def anoAposEleicao(ano):
    if ano == 2008:
        return 2009
    elif ano == 2012:
        return 2013
    
path_ges, path_jul  = r'~/Downloads/codigo_ugestora.csv', r'~/Downloads/contas_julgadas_irregulares_TRE_2016.csv'
ud_gestoras, contas_jul_irr = pd.read_csv(path_ges), pd.read_csv(path_jul)

#Filtro por ano entre 2008 e 2016
ud_gestoras = ud_gestoras[ud_gestoras['dt_Ano']>2008]

#Solucao do ano
vfunc = np.vectorize(mudaAnoUgestora)
anos = vfunc(ud_gestoras['dt_Ano'].values)
ud_gestoras['dt_Ano'] = anos

#Solucao do nome da gestora
vfunc = np.vectorize(nomeUgestora)
ud_gestoras_pref = ud_gestoras[ud_gestoras['de_Ugestora'].str.contains("Prefeitura")]
municipios = vfunc(ud_gestoras_pref['de_Ugestora'].values)
ud_gestoras_pref['de_Ugestora'] = municipios
ud_gestoras_pref = ud_gestoras_pref[['cd_Ugestora','dt_Ano','de_Ugestora']]

path_cand_2008, path_cand_2012 = r'~/Downloads/candidate_local_2008-1.csv', r'~/Downloads/candidate_local_2012-1.csv'

#para 2008
cand_2008 = pd.read_csv(path_cand_2008)
cand_2008_PB = cand_2008[(cand_2008['SIGLA_UF'] == 'PB')]
cand_2008_PB = cand_2008_PB[(cand_2008_PB['CODIGO_CARGO'] == 11)]
cand_2008_PB_ELEIT = cand_2008_PB[(cand_2008_PB['DESC_SIT_TOT_TURNO'] == 'ELEITO')]
cand_2008_PB_ELEIT = cand_2008_PB_ELEIT[['ANO_ELEICAO','DESCRICAO_UE', 'NOME_CANDIDATO']]

#para 2012
cand_2012 = pd.read_csv(path_cand_2012)
cand_2012_PB = cand_2012[(cand_2012['SIGLA_UF'] == 'PB')]
cand_2012_PB = cand_2012_PB[(cand_2012_PB['CODIGO_CARGO'] == 11)]
cand_2012_PB_ELEIT = cand_2012_PB[(cand_2012_PB['DESC_SIT_TOT_TURNO'] == 'ELEITO')]
cand_2012_PB_ELEIT = cand_2012_PB_ELEIT[['ANO_ELEICAO','DESCRICAO_UE', 'NOME_CANDIDATO']]

#concatenação de 2008 e 2012
eleicoes2008_2012 = [cand_2008_PB_ELEIT,cand_2012_PB_ELEIT]
result = pd.concat(eleicoes2008_2012)

vfunc = np.vectorize(anoAposEleicao)
anos = vfunc(result['ANO_ELEICAO'].values)
result['ANO_ELEICAO'] = anos

result.rename(columns={'DESCRICAO_UE': 'de_Ugestora', 'ANO_ELEICAO':'dt_Ano'}, inplace=True)
final = pd.merge(ud_gestoras_pref, result, on=['de_Ugestora', 'dt_Ano'])
final = final.drop_duplicates()
final.rename(columns={'NOME_CANDIDATO': 'ELEITO'}, inplace=True)

#Sobre a tabela do TCE
contas_jul_irr.rename(columns={'JURISDICIONADO': 'de_Ugestora'}, inplace=True)

#Corrige ano para xx/mm/yyyy 
vfunc = np.vectorize(anoPublicacao)
anos = vfunc(contas_jul_irr['DATA PUBLICAÇÃO'].values)
contas_jul_irr['DATA PUBLICAÇÃO'] = anos

# Filtra para anos maiores que 2008.
contas_jul_irr = contas_jul_irr[contas_jul_irr['DATA PUBLICAÇÃO'] > 2008]
contas_jul_irr.rename(columns={'DATA PUBLICAÇÃO': 'dt_Ano'}, inplace=True)

vfunc = np.vectorize(upperPrefeito)
responsaveis = vfunc(contas_jul_irr['RESPONSÁVEL'].values)
contas_jul_irr['RESPONSÁVEL'] = responsaveis
eleitos = vfunc(final['ELEITO'].values)
final['ELEITO'] = eleitos

vfunc = np.vectorize(nomeUgestora)
contas_jul = contas_jul_irr[contas_jul_irr['de_Ugestora'].str.contains("Prefeitura")]
municipios = vfunc(contas_jul['de_Ugestora'].values)
contas_jul['de_Ugestora'] = municipios

all_final = pd.merge(contas_jul, final, on=['de_Ugestora','dt_Ano'])
all_final.to_csv('~/Desktop/TRE_Sagres_Resp_Eleito.csv',sep=',',encoding='utf-8', index=False)

/Users/zegildo/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zegildo/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ITEM,PROCESSO,SUBCATEGORIA,de_Ugestora,CPF,RESPONSÁVEL,DECISÃO,dt_Ano,RES. DECISÃO PODER LEGISLATIVO,cd_Ugestora,ELEITO
0,10,01910/08,PCA - Prestação de Contas Anuais,SERRA DA RAIZ,409.573.904-59,ADAILMA FERNANDES DA SILVA,PPL-TC 00157/10,2009,NaN,201202,LUIZ GONZAGA BEZERRA DUARTE
1,11,03196/09,PCA - Prestação de Contas Anuais,SERRA DA RAIZ,409.573.904-59,ADAILMA FERNANDES DA SILVA,PPL-TC 00118/10,2009,"Dec. Leg. 004/2011, rejeitando o Parecer",201202,LUIZ GONZAGA BEZERRA DUARTE
2,35,05354/13,PCA - Prestação de Contas Anuais,MALTA,001.214.504-14,AJACIO GOMES WANDERLEY,APL-TC 00833/13,2013,"Dec. Leg. 001/2014, mantendo o Parecer",201109,MANOEL BENEDITO DE LUCENA FILHO
3,36,05354/13,PCA - Prestação de Contas Anuais,MALTA,001.214.504-14,AJACIO GOMES WANDERLEY,PPL-TC 00205/13,2013,"Dec. Leg. 001/2014, mantendo o Parecer",201109,MANOEL BENEDITO DE LUCENA FILHO
4,57,02303/08,PCA - Prestação de Contas Anuais,FREI MARTINHO,NaN,ANA ADELIA NERY CABRAL,APL-TC 00259/12,2009,NaN,201080,FRANCIVALDO SANTOS DE ARAUJO
5,58,02303/08,PCA - Prestação de Contas Anuais,FREI MARTINHO,NaN,ANA ADELIA NERY CABRAL,PPL-TC 00058/12,2009,NaN,201080,FRANCIVALDO SANTOS DE ARAUJO
6,59,02543/07,PCA - Prestação de Contas Anuais,FREI MARTINHO,NaN,ANA ADELIA NERY CABRAL,PPL-TC 00105/09,2009,NaN,201080,FRANCIVALDO SANTOS DE ARAUJO
7,60,03171/09,PCA - Prestação de Contas Anuais,FREI MARTINHO,NaN,ANA ADELIA NERY CABRAL,APL-TC 01005/11,2009,"Dec. Leg. 001/2013, mantendo o Parecer",201080,FRANCIVALDO SANTOS DE ARAUJO
8,61,03171/09,PCA - Prestação de Contas Anuais,FREI MARTINHO,NaN,ANA ADELIA NERY CABRAL,PPL-TC 00239/11,2009,"Dec. Leg. 001/2013, mantendo o Parecer",201080,FRANCIVALDO SANTOS DE ARAUJO
9,76,02942/09,PCA - Prestação de Contas Anuais,UMBUZEIRO,059.805.184-87,ANTONIO FERNANDES DE LIMA,PPL-TC 00022/11,2009,NaN,201219,ANTONIO FERNANDES DE LIMA


In [235]:
all_final.to_csv('~/Desktop/TRE_Sagres_Resp_Eleito.csv',sep=',',encoding='utf-8', index=False)

In [253]:
#final['ELEITO'].unique()
idon = final[~(final['ELEITO'].isin(all_final['ELEITO'].unique()))]
idon.to_csv('~/Desktop/TRE_Sagres_Eleit_Idon.csv',sep=',',encoding='utf-8', index=False)
#idon_n = idon['ELEITO'].unique()
#corrp_n = all_final['ELEITO'].unique()
#intersec = set(idon_n).intersection(corrp_n)
#print intersec